In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('/Users/ling/Desktop/ds_class/DS-SF-42/final_project/kag_risk_factors_cervical_cancer.csv')

## 1.Load in dataset and do some EDA work. 

1. EDA Work 
- cleaning - replace NANS. turn into numeric. 
- how much data is there
- how much NAs is there. if a column has a lot, drop? 
- what is bad/good
- how much do i want to use

2. Analysis without modeling
- corr of variables
- map out counts of each 


2. Models - be clear about what i want to determine
- Log regression
- Decision Tree
- Random Forest
- Adaboost 
- calculate confusion matrix and interepret it 



Notes from Kaggle: 
Positive exams results doesn’t necessarily imply in a diagnostic, but as multiple exams return positive, the greater the likelyhood of cervical cancer.
- what is null accuracy - 88%
- calculate confusion matrix and interepret it (KERAs notebook) 



Questions
- If nan, should we just ignore, or can we fill with a median value? 

In [9]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 858 entries, 0 to 857
Data columns (total 36 columns):
Age                                   858 non-null int64
Number of sexual partners             858 non-null object
First sexual intercourse              858 non-null object
Num of pregnancies                    858 non-null object
Smokes                                858 non-null object
Smokes (years)                        858 non-null object
Smokes (packs/year)                   858 non-null object
Hormonal Contraceptives               858 non-null object
Hormonal Contraceptives (years)       858 non-null object
IUD                                   858 non-null object
IUD (years)                           858 non-null object
STDs                                  858 non-null object
STDs (number)                         858 non-null object
STDs:condylomatosis                   858 non-null object
STDs:cervical condylomatosis          858 non-null object
STDs:vaginal condylomatosi